In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
json_path = 'datatran_consolidado.json'

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

sns.set_style("whitegrid")
# Aumentar o tamanho padrão dos gráficos
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Célula 2: Carregamento e Inspeção Inicial de Dados

print("--- 1. Carregamento e Limpeza Inicial de Dados ---")

NameError: name 'json' is not defined

In [ ]:
try:
    df = pd.read_json('datatran_consolidado.json')
    print("\nDataFrame carregado com sucesso!")
except Exception as e:
    print(f"Erro ao carregar o arquivo: {e}")
    print("Verifique se o arquivo 'datatran_consolidado.json' está na mesma pasta do seu Notebook.")
    # Exibir apenas as primeiras linhas se o carregamento for bem-sucedido
    if 'df' in locals():
        print("\nPrimeiras 5 linhas do DataFrame:")
        display(df.head())

print("\nInformações sobre o DataFrame:")
df.info()

In [ ]:
print("--- 2. Análise de Frequência ---")

# Gráfico 1: Acidentes por UF (Top 10)
plt.figure(figsize=(12, 6))
df['uf'].value_counts().nlargest(10).sort_values(ascending=False).plot(
    kind='bar',
    color=sns.color_palette("viridis")
)
plt.title('Top 10 Estados (UF) com Mais Acidentes', fontsize=16)
plt.xlabel('UF (Estado)', fontsize=14)
plt.ylabel('Número de Acidentes', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
ordem_dias = ['segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira', 'sábado', 'domingo']
df['dia_semana'].value_counts().reindex(ordem_dias, fill_value=0).plot(
    kind='bar',
    color=sns.color_palette("plasma", len(ordem_dias))
)
plt.title('Acidentes por Dia da Semana', fontsize=16)
plt.xlabel('Dia da Semana', fontsize=14)
plt.ylabel('Número de Acidentes', fontsize=14)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
print("--- 3. Análise Temporal ---")

# Extrair a hora do campo 'horario'
df['hora'] = df['horario'].str.split(':').str[0].astype(int)

# Contar e ordenar por hora
frequencia_hora = df['hora'].value_counts().sort_index()

# Gráfico 3: Distribuição de Acidentes por Hora do Dia
plt.figure(figsize=(12, 6))
frequencia_hora.plot(
    kind='line',
    marker='o',
    color='darkblue'
)
plt.title('Distribuição de Acidentes por Hora do Dia', fontsize=16)
plt.xlabel('Hora do Dia (0 a 23)', fontsize=14)
plt.ylabel('Número de Acidentes', fontsize=14)
plt.xticks(range(0, 24, 2)) # Marcas a cada 2 horas
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

print(f"\nA hora de pico de acidentes é a hora {frequencia_hora.idxmax()} com {frequencia_hora.max()} ocorrências.")


In [ ]:
print("--- 4. Análise de Correlação (Heatmap) ---")

# Tabela de Contingência (Frequência Cruzada) para correlação entre Condição Metereológica e Tipo de Acidente
# Normalizar os dados por linha para ver a PROPORÇÃO (probabilidade condicional)
contingency_table = pd.crosstab(
    df['condicao_metereologica'],
    df['tipo_acidente'],
    normalize='index'
) * 100

# Filtrar para os Top 5 Tipos de Acidente para manter o Heatmap legível
top_5_tipos = df['tipo_acidente'].value_counts().nlargest(5).index
contingency_table_filtered = contingency_table[top_5_tipos]

print("\nTabela de Proporção (%) - Tipo de Acidente vs. Condição Metereológica (Linhas normalizadas):")
display(contingency_table_filtered.round(2))

# Gráfico 4: Heatmap da Correlação (Proporção)
plt.figure(figsize=(12, 8))
sns.heatmap(
    contingency_table_filtered,
    annot=True,        # Mostrar os valores na matriz
    fmt=".1f",         # Formato com uma casa decimal
    cmap="YlGnBu",     # Paleta de cores
    linewidths=.5,
    cbar_kws={'label': 'Proporção (%)'} # Rótulo para a barra de cores
)
plt.title('Correlação (Proporção) entre Condição Metereológica e Tipos de Acidente', fontsize=16)
plt.xlabel('Tipo de Acidente', fontsize=14)
plt.ylabel('Condição Metereológica', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()